In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st
import spotipy 

# Cargar variables de entorno desde un archivo .env
load_dotenv()

# Importar funciones de gestión
from utils.chat_gpt_manager import (
    get_chords_chat,
    create_song
)
from utils.spotify_manager import (
    get_similar_songs,
    get_audio_analysis,
    get_audio_features,
    get_song_and_artist
)
from utils.functions import (
    add_info_list_df,
    convert_chords_with_minor,
    standardize_chord_symbols,
    transpose_chords_to_key,
    check_track_csv,
    filter_existing_vector_id,
    to_embbed_text_column,
    expand_and_remove_original_columns,
    drop_columns_with_many_nulls,
    get_user_input,
    handle_input_search,
    calc_diff,
    export_unique_rows,
    create_metadata_column,
    clean_metadata_column,
    insert_into_pinecone,
    create_dataframe_for_pinecone,
    realizar_consulta

    
)

from utils.pinecone_manager import (
    initialize_pinecone, 
    setup_index
)

from utils.lyrics_manager import insert_lyrics_db

from pinecone import Pinecone, ServerlessSpec
from pinecone.grpc import PineconeGRPC as Pinecone

# Initizalize pinecone

In [2]:
pc = initialize_pinecone()

In [3]:
index = setup_index(pc)

Index 'song-starter-index' already exists, skipping creation.


In [4]:
index_name= pc.list_indexes().indexes[0].name

# API de Spotify

Aquí se utiliza la API de Spotify para obtener recomendaciones de canciones basadas en una canción dada. 

Input:

In [5]:
input_song, input_artist = get_user_input()

In [6]:
# Llamar a la función para obtener el track ID
input_track_id=handle_input_search(input_song, input_artist)

In [7]:
input_song , input_artist = get_song_and_artist(input_track_id)

In [8]:
input_data = pd.DataFrame({'id': [input_track_id]})
input_data['name'] = input_song
input_data['artists'] = input_artist

In [9]:
input_data

,id,name,artists
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira


Similar songs

In [10]:
tracks_data = get_similar_songs(input_track_id, 5)

In [11]:
for track in tracks_data:
    print(track['name'])


Un Poco de Amor
La Fiesta
Hoy Tengo Ganas De Ti
Amigos (con Maria Becerra)
No Pasa Nada


In [12]:
tracks_data = pd.DataFrame(tracks_data)

In [13]:
tracks_data.head(2)

,id,name,artists
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira
1,2o6pi3nx1UAYxuBccabCMQ,La Fiesta,Pedro Capó


MIRAR ESTO, cómo evitar que a partir de aquí todo de error si ya están todas en pinecone

In [14]:
filter_existing_vector_id(index, tracks_data)            

In [15]:

tracks_data.reset_index(drop=True, inplace=True)

In [16]:
tracks_data

,id,name,artists
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra"
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash


In [17]:
csv= pd.read_csv('mi_archivo_temporal.csv')

In [18]:
if len(tracks_data)>0:
    df=check_track_csv(tracks_data, csv)

In [19]:
if len(df)>0:    
    df.reset_index(drop=True, inplace=True)

# API de ChatGPT

Aquí se utiliza la API de Chat GPT, para obtener los acordes e instrumentos de las canciones

Input:

In [20]:
add_info_list_df(input_data, get_chords_chat, 'chords', max_retries=3)

,id,name,artists,chords
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]"


In [21]:
add_info_list_df(input_data, get_chords_chat, 'instruments', max_retries=3)

,id,name,artists,chords,instruments
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]"


Related tracks:

In [22]:
add_info_list_df(df, get_chords_chat, 'chords', max_retries=3)

,id,name,artists,chords
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, G, D, Em, C]"
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, Em, F, G]"
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, G, D, Em, C]"


In [23]:
add_info_list_df(df, get_chords_chat, 'instruments', max_retries=3)

,id,name,artists,chords,instruments
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, G, D, Em, C]","[C, G, Am, F, C, Em, F, G]"
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, Em, F, G]","[C, G, Am, F, C, G, Am, F]"
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, G, D, Em, C]","[C, G, Am, F, D, A, Bm, G]"


# API de Spotify para nuevas columnas

Input:

In [24]:
get_audio_features(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0


In [25]:
get_audio_analysis(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0,1,0,108.977,-4.767


In [26]:
insert_lyrics_db(input_data)

(                       id                         name  artists  \
 0  3TY1PFZXmYeiLcXygB74My  Suerte (Whenever, Wherever)  Shakira   
 
                       chords                instruments  speechiness  \
 0  [C, G, Am, F, C, G, F, F]  [D, A, Bm, G, D, A, G, A]       0.0337   
 
    instrumentalness  key  mode    tempo  loudness  \
 0                 0    1     0  108.977    -4.767   
 
                                                theme  
 0  Suerte que en el sur hayas nacido\nY que burle...  ,
 'Suerte que en el sur hayas nacido\nY que burlemos las distancias,\nSuerte que es haberte conocido\nY por ti amar tierras extrañas\nYo puedo escalar los andes solo\nPor ir a contar tus lunares,\n\nContigo celebro y sufro todo')

In [27]:
input_data['chords'].dropna(inplace=True)
input_data = input_data[input_data['chords'].apply(lambda x: x != [])]

In [28]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...


Related tracks:

In [29]:
get_audio_features(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, G, D, Em, C]","[C, G, Am, F, C, Em, F, G]",0.0412,0.000007
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, Em, F, G]","[C, G, Am, F, C, G, Am, F]",0.0450,0.000000
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, G, D, Em, C]","[C, G, Am, F, D, A, Bm, G]",0.0306,0.000000


In [30]:
get_audio_analysis(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, G, D, Em, C]","[C, G, Am, F, C, Em, F, G]",0.0412,0.000007,11,1,101.451,-9.947
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, Em, F, G]","[C, G, Am, F, C, G, Am, F]",0.0450,0.000000,11,0,157.002,-3.459
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, G, D, Em, C]","[C, G, Am, F, D, A, Bm, G]",0.0306,0.000000,0,1,136.145,-5.068


In [31]:
insert_lyrics_db(df)

Timeout o fallo! Usando GPT para obtener el tema para Pablo Alborán, Maria Becerra - Amigos (con Maria Becerra)


(                       id                        name  \
 0  6dwhYg12fOFw8aUrnCOOqC             Un Poco de Amor   
 1  20NeR3CwuyGfRktTncKRU7  Amigos (con Maria Becerra)   
 2  6Gusd5o0sQD1jmmoQlvNw4                No Pasa Nada   
 
                         artists                      chords  \
 0                       Shakira  [C, G, Am, F, G, D, Em, C]   
 1  Pablo Alborán, Maria Becerra  [C, G, Am, F, C, Em, F, G]   
 2                        Ha*Ash  [C, G, Am, F, G, D, Em, C]   
 
                   instruments  speechiness  instrumentalness  key  mode  \
 0  [C, G, Am, F, C, Em, F, G]       0.0412          0.000007   11     1   
 1  [C, G, Am, F, C, G, Am, F]       0.0450          0.000000   11     0   
 2  [C, G, Am, F, D, A, Bm, G]       0.0306          0.000000    0     1   
 
      tempo  loudness                                              theme  
 0  101.451    -9.947  Rasta rootsme again so come listen to one \nth...  
 1  157.002    -3.459  The song "Amigos" by Pablo Al

In [32]:
df_vectorial= df.copy()
df_vectorial['chords'].dropna(inplace=True)
df_vectorial = df_vectorial[df_vectorial['chords'].apply(lambda x: x != [])]

# Separate columns

Input:

In [33]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...


In [34]:
input_data['chords_1'] = input_data['chords'].apply(lambda x: x[:4]) 
input_data['chords_2'] = input_data['chords'].apply(lambda x: x[4:])

In [35]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...,"[C, G, Am, F]","[C, G, F, F]"


In [36]:
input_data['chords_1'] = input_data['chords_1'].apply(standardize_chord_symbols)
input_data['chords_2'] = input_data['chords_2'].apply(standardize_chord_symbols)

In [37]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...,"[C, G, Am, F]","[C, G, F, F]"


In [38]:
input_data['chords_1_numeric'] = input_data['chords_1'].apply(convert_chords_with_minor)
input_data['chords_2_numeric'] = input_data['chords_2'].apply(convert_chords_with_minor)

In [39]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...,"[C, G, Am, F]","[C, G, F, F]","[0, 7, 21, 5]","[0, 7, 5, 5]"


In [40]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    input_data[f'normal_{col}'] = input_data.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...,"[C, G, Am, F]","[C, G, F, F]","[0, 7, 21, 5]","[0, 7, 5, 5]","[-1, 6, 20, 4]","[-1, 6, 4, 4]"


In [41]:
for col in col_chords:
    input_data[f'diff_{col}'] = input_data.apply(lambda row: calc_diff(row[col]), axis=1)

input_data


,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...,"[C, G, Am, F]","[C, G, F, F]","[0, 7, 21, 5]","[0, 7, 5, 5]","[-1, 6, 20, 4]","[-1, 6, 4, 4]","[7, 14, -16]","[7, -2, 0]"


In [42]:
input_data.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [43]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido Y que burlem...,"[C, G, Am, F]","[C, G, F, F]","[0, 7, 21, 5]","[0, 7, 5, 5]","[-1, 6, 20, 4]","[-1, 6, 4, 4]","[7, 14, -16]","[7, -2, 0]"


Related tracks:

In [44]:
df_vectorial['chords_1'] = df_vectorial['chords'].apply(lambda x: x[:4]) 
df_vectorial['chords_2'] = df_vectorial['chords'].apply(lambda x: x[4:])
df_vectorial.drop(columns=['chords'], inplace=True)

In [45]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, C, Em, F, G]",0.0412,0.000007,11,1,101.451,-9.947,Rasta rootsme again so come listen to one \nth...,"[C, G, Am, F]","[G, D, Em, C]"
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, G, Am, F]",0.0450,0.000000,11,0,157.002,-3.459,"The song ""Amigos"" by Pablo Alborán featuring M...","[C, G, Am, F]","[C, Em, F, G]"
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, D, A, Bm, G]",0.0306,0.000000,0,1,136.145,-5.068,"Me gustaste por decente, irreverente, por inte...","[C, G, Am, F]","[G, D, Em, C]"


In [46]:
# Nos ocupamos de bemoles o sostenidos para estandarizarlos
df_vectorial['chords_1'] = df_vectorial['chords_1'].apply(standardize_chord_symbols)
df_vectorial['chords_2'] = df_vectorial['chords_2'].apply(standardize_chord_symbols)

In [47]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, C, Em, F, G]",0.0412,0.000007,11,1,101.451,-9.947,Rasta rootsme again so come listen to one \nth...,"[C, G, Am, F]","[G, D, Em, C]"
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, G, Am, F]",0.0450,0.000000,11,0,157.002,-3.459,"The song ""Amigos"" by Pablo Alborán featuring M...","[C, G, Am, F]","[C, Em, F, G]"
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, D, A, Bm, G]",0.0306,0.000000,0,1,136.145,-5.068,"Me gustaste por decente, irreverente, por inte...","[C, G, Am, F]","[G, D, Em, C]"


In [48]:
# Convertimos los acordes a valores numéricos
df_vectorial['chords_1_numeric'] = df_vectorial['chords_1'].apply(convert_chords_with_minor)
df_vectorial['chords_2_numeric'] = df_vectorial['chords_2'].apply(convert_chords_with_minor)

In [49]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, C, Em, F, G]",0.0412,0.000007,11,1,101.451,-9.947,Rasta rootsme again so come listen to one \nth...,"[C, G, Am, F]","[G, D, Em, C]","[0, 7, 21, 5]","[7, 2, 16, 0]"
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, G, Am, F]",0.0450,0.000000,11,0,157.002,-3.459,"The song ""Amigos"" by Pablo Alborán featuring M...","[C, G, Am, F]","[C, Em, F, G]","[0, 7, 21, 5]","[0, 16, 5, 7]"
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, D, A, Bm, G]",0.0306,0.000000,0,1,136.145,-5.068,"Me gustaste por decente, irreverente, por inte...","[C, G, Am, F]","[G, D, Em, C]","[0, 7, 21, 5]","[7, 2, 16, 0]"


In [50]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    df_vectorial[f'normal_{col}'] = df_vectorial.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, C, Em, F, G]",0.0412,0.000007,11,1,101.451,-9.947,Rasta rootsme again so come listen to one \nth...,"[C, G, Am, F]","[G, D, Em, C]","[0, 7, 21, 5]","[7, 2, 16, 0]","[-11, -4, 10, -6]","[-4, -9, 5, -11]"
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, G, Am, F]",0.0450,0.000000,11,0,157.002,-3.459,"The song ""Amigos"" by Pablo Alborán featuring M...","[C, G, Am, F]","[C, Em, F, G]","[0, 7, 21, 5]","[0, 16, 5, 7]","[-11, -4, 10, -6]","[-11, 5, -6, -4]"
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, D, A, Bm, G]",0.0306,0.000000,0,1,136.145,-5.068,"Me gustaste por decente, irreverente, por inte...","[C, G, Am, F]","[G, D, Em, C]","[0, 7, 21, 5]","[7, 2, 16, 0]","[0, 7, 21, 5]","[7, 2, 16, 0]"


In [51]:
for col in col_chords:
    df_vectorial[f'diff_{col}'] = df_vectorial.apply(lambda row: calc_diff(row[col]), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, C, Em, F, G]",0.0412,0.000007,11,1,101.451,-9.947,Rasta rootsme again so come listen to one \nth...,"[C, G, Am, F]","[G, D, Em, C]","[0, 7, 21, 5]","[7, 2, 16, 0]","[-11, -4, 10, -6]","[-4, -9, 5, -11]","[7, 14, -16]","[-5, 14, -16]"
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, G, Am, F]",0.0450,0.000000,11,0,157.002,-3.459,"The song ""Amigos"" by Pablo Alborán featuring M...","[C, G, Am, F]","[C, Em, F, G]","[0, 7, 21, 5]","[0, 16, 5, 7]","[-11, -4, 10, -6]","[-11, 5, -6, -4]","[7, 14, -16]","[16, -11, 2]"
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, D, A, Bm, G]",0.0306,0.000000,0,1,136.145,-5.068,"Me gustaste por decente, irreverente, por inte...","[C, G, Am, F]","[G, D, Em, C]","[0, 7, 21, 5]","[7, 2, 16, 0]","[0, 7, 21, 5]","[7, 2, 16, 0]","[7, 14, -16]","[-5, 14, -16]"


In [52]:
df_vectorial.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [53]:
df_vectorial = df_vectorial.fillna(0)

In [54]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, C, Em, F, G]",0.0412,0.000007,11,1,101.451,-9.947,Rasta rootsme again so come listen to one thi...,"[C, G, Am, F]","[G, D, Em, C]","[0, 7, 21, 5]","[7, 2, 16, 0]","[-11, -4, 10, -6]","[-4, -9, 5, -11]","[7, 14, -16]","[-5, 14, -16]"
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, G, Am, F]",0.0450,0.000000,11,0,157.002,-3.459,"The song ""Amigos"" by Pablo Alborán featuring M...","[C, G, Am, F]","[C, Em, F, G]","[0, 7, 21, 5]","[0, 16, 5, 7]","[-11, -4, 10, -6]","[-11, 5, -6, -4]","[7, 14, -16]","[16, -11, 2]"
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, D, A, Bm, G]",0.0306,0.000000,0,1,136.145,-5.068,"Me gustaste por decente, irreverente, por inte...","[C, G, Am, F]","[G, D, Em, C]","[0, 7, 21, 5]","[7, 2, 16, 0]","[0, 7, 21, 5]","[7, 2, 16, 0]","[7, 14, -16]","[-5, 14, -16]"


In [55]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido Y que burlem...,"[C, G, Am, F]","[C, G, F, F]","[0, 7, 21, 5]","[0, 7, 5, 5]","[-1, 6, 20, 4]","[-1, 6, 4, 4]","[7, 14, -16]","[7, -2, 0]"


# Backup csv

In [56]:
csv_file = 'mi_archivo_temporal.csv'

# Llamar la función para exportar
export_unique_rows(df_vectorial, csv_file)



/Users/gabrielacaseromonge/Documents/GitHub/song-sarter-streamlit/utils/functions.py:277: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: str(x) if isinstance(x, list) else x)
/Users/gabrielacaseromonge/Documents/GitHub/song-sarter-streamlit/utils/functions.py:283: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') and x.endswith(']') else x)


# Expand columns

In [57]:
# Especificar las columnas que contienen listas (excepto 'embedding')
columns_to_expand = ['instruments', 'chords_1', 'chords_2', 
                      'chords_1_numeric', 'chords_2_numeric', 
                      'normal_chords_1_numeric', 'normal_chords_2_numeric', 
                      'diff_chords_1_numeric', 'diff_chords_2_numeric']

Input:

In [58]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido Y que burlem...,"[C, G, Am, F]","[C, G, F, F]","[0, 7, 21, 5]","[0, 7, 5, 5]","[-1, 6, 20, 4]","[-1, 6, 4, 4]","[7, 14, -16]","[7, -2, 0]"


In [59]:
columns_to_expand

['instruments',
 'chords_1',
 'chords_2',
 'chords_1_numeric',
 'chords_2_numeric',
 'normal_chords_1_numeric',
 'normal_chords_2_numeric',
 'diff_chords_1_numeric',
 'diff_chords_2_numeric']

In [60]:
input_expanded = expand_and_remove_original_columns(input_data, columns_to_expand)

In [61]:
input_expanded

,id,name,artists,chords,speechiness,instrumentalness,key,mode,tempo,loudness,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]",0.0337,0,1,0,108.977,-4.767,...,-1,6,4,4,7,14,-16,7,-2,0


In [62]:
#elimino las columnas que tengan solo valores nulos
input_expanded = input_expanded.dropna(axis=1, how='all')

Related tracks:

In [63]:
df_vectorial_expanded = expand_and_remove_original_columns(df_vectorial, columns_to_expand)

In [64]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,0.0412,0.000007,11,1,101.451,-9.947,Rasta rootsme again so come listen to one thi...,...,-4,-9,5,-11,7,14,-16,-5,14,-16
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra",0.0450,0.000000,11,0,157.002,-3.459,"The song ""Amigos"" by Pablo Alborán featuring M...",...,-11,5,-6,-4,7,14,-16,16,-11,2
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,0.0306,0.000000,0,1,136.145,-5.068,"Me gustaste por decente, irreverente, por inte...",...,7,2,16,0,7,14,-16,-5,14,-16


In [65]:
#elimino las columnas que tengan solo valores nulos
df_vectorial_expanded = df_vectorial_expanded.dropna(axis=1, how='all')

In [66]:
drop_columns_with_many_nulls(df_vectorial_expanded, 15)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,0.0412,0.000007,11,1,101.451,-9.947,Rasta rootsme again so come listen to one thi...,...,-4,-9,5,-11,7,14,-16,-5,14,-16
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra",0.0450,0.000000,11,0,157.002,-3.459,"The song ""Amigos"" by Pablo Alborán featuring M...",...,-11,5,-6,-4,7,14,-16,16,-11,2
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,0.0306,0.000000,0,1,136.145,-5.068,"Me gustaste por decente, irreverente, por inte...",...,7,2,16,0,7,14,-16,-5,14,-16


In [67]:
df_vectorial_expanded.head(3)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,0.0412,0.000007,11,1,101.451,-9.947,Rasta rootsme again so come listen to one thi...,...,-4,-9,5,-11,7,14,-16,-5,14,-16
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra",0.0450,0.000000,11,0,157.002,-3.459,"The song ""Amigos"" by Pablo Alborán featuring M...",...,-11,5,-6,-4,7,14,-16,16,-11,2
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,0.0306,0.000000,0,1,136.145,-5.068,"Me gustaste por decente, irreverente, por inte...",...,7,2,16,0,7,14,-16,-5,14,-16


# Create column for the text to create the embeddings

Input:

In [68]:
input_expanded=to_embbed_text_column(input_expanded)

In [69]:
input_expanded['to_embbed_text']

0    name: Suerte (Whenever, Wherever) artists: Sha...
Name: to_embbed_text, dtype: object

Related songs:

In [70]:
df_vectorial_expanded=to_embbed_text_column(df_vectorial_expanded)

In [71]:
df_vectorial_expanded


,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,0.0412,0.000007,11,1,101.451,-9.947,Rasta rootsme again so come listen to one thi...,...,-9,5,-11,7,14,-16,-5,14,-16,name: Un Poco de Amor artists: Shakira speechi...
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra",0.0450,0.000000,11,0,157.002,-3.459,"The song ""Amigos"" by Pablo Alborán featuring M...",...,5,-6,-4,7,14,-16,16,-11,2,name: Amigos (con Maria Becerra) artists: Pabl...
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,0.0306,0.000000,0,1,136.145,-5.068,"Me gustaste por decente, irreverente, por inte...",...,2,16,0,7,14,-16,-5,14,-16,name: No Pasa Nada artists: Ha*Ash speechiness...


# Text for embeddings to chat gpt

In [72]:
api_key = os.getenv("OPENAI_API_KEY")

# Configurar la clave de API en OpenAI
OpenAI.api_key = api_key

In [73]:
# Convert text into vectors using embeddings
client = OpenAI(
 api_key=api_key,
)

Input:

In [74]:
input_expanded['embedding'] = input_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [75]:
input_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [76]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,0.0412,0.000007,11,1,101.451,-9.947,Rasta rootsme again so come listen to one thi...,...,-9,5,-11,7,14,-16,-5,14,-16,name: Un Poco de Amor artists: Shakira speechi...
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra",0.0450,0.000000,11,0,157.002,-3.459,"The song ""Amigos"" by Pablo Alborán featuring M...",...,5,-6,-4,7,14,-16,16,-11,2,name: Amigos (con Maria Becerra) artists: Pabl...
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,0.0306,0.000000,0,1,136.145,-5.068,"Me gustaste por decente, irreverente, por inte...",...,2,16,0,7,14,-16,-5,14,-16,name: No Pasa Nada artists: Ha*Ash speechiness...


Related songs:

In [77]:
#para cada fila del dataframe, sacar el vector de embedding de la columna "to_embbed_text"
df_vectorial_expanded['embedding'] = df_vectorial_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [78]:
df_vectorial_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [79]:
df_vectorial_expanded = df_vectorial_expanded.rename(columns={'embedding': 'values'})

# Data to pinecone

In [80]:

# Procesar el DataFrame
df_vectorial_expanded = create_metadata_column(df_vectorial_expanded)
df_vectorial_expanded = clean_metadata_column(df_vectorial_expanded)
df_para_pinecone = create_dataframe_for_pinecone(df_vectorial_expanded)
insert_into_pinecone(df_para_pinecone, index)



sending upsert requests:   0%|          | 0/3 [00:00<?, ?it/s]

In [81]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,values,metadata
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,0.0412,0.000007,11,1,101.451,-9.947,Rasta rootsme again so come listen to one thi...,...,5,-11,7,14,-16,-5,14,-16,"[0.007981328293681145, -0.018507039174437523, ...","{'name': 'Un Poco de Amor', 'artists': 'Shakir..."
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra",0.0450,0.000000,11,0,157.002,-3.459,"The song ""Amigos"" by Pablo Alborán featuring M...",...,-6,-4,7,14,-16,16,-11,2,"[-0.0036139357835054398, 0.008265986107289791,...","{'name': 'Amigos (con Maria Becerra)', 'artist..."
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,0.0306,0.000000,0,1,136.145,-5.068,"Me gustaste por decente, irreverente, por inte...",...,16,0,7,14,-16,-5,14,-16,"[0.019795119762420654, -0.013124878518283367, ...","{'name': 'No Pasa Nada', 'artists': 'Ha*Ash', ..."


In [82]:
df_para_pinecone

,id,values,metadata
0,6dwhYg12fOFw8aUrnCOOqC,"[0.007981328293681145, -0.018507039174437523, ...","{'name': 'Un Poco de Amor', 'artists': 'Shakir..."
1,20NeR3CwuyGfRktTncKRU7,"[-0.0036139357835054398, 0.008265986107289791,...","{'name': 'Amigos (con Maria Becerra)', 'artist..."
2,6Gusd5o0sQD1jmmoQlvNw4,"[0.019795119762420654, -0.013124878518283367, ...","{'name': 'No Pasa Nada', 'artists': 'Ha*Ash', ..."


# Query search in pinecone

In [83]:
response = realizar_consulta(client, input_expanded, index, top_k=5)

Resultado de la consulta: {'matches': [{'id': '1rqGuE3jvcvacQk96lfnl6',
              'metadata': {'artists': 'Geese',
                           'chords_1_1': 'G',
                           'chords_1_2': 'D',
                           'chords_1_3': 'Em',
                           'chords_1_4': 'C',
                           'chords_1_numeric_1': 7.0,
                           'chords_1_numeric_2': 2.0,
                           'chords_1_numeric_3': 16.0,
                           'chords_1_numeric_4': 0.0,
                           'chords_2_1': 'Am',
                           'chords_2_2': 'F',
                           'chords_2_3': 'C',
                           'chords_2_4': 'G',
                           'chords_2_numeric_1': 21.0,
                           'chords_2_numeric_2': 5.0,
                           'chords_2_numeric_3': 0.0,
                           'chords_2_numeric_4': 7.0,
                           'diff_chords_1_numeric_1': -5.0,
                 

In [84]:
response_matches = response['matches']

details = []
for match in response_matches:
    details.append(f"Intruments: {match['metadata']['instruments_1']} {match['metadata']['instruments_2']}, Tempo: {match['metadata']['tempo']}, Speechiness: {match['metadata']['speechiness']}, Loudness: {match['metadata']['loudness']}")

In [85]:
details

['Intruments: guitar bass, Tempo: 131.948, Speechiness: 0.0382, Loudness: -7.699',
 'Intruments: piano cello, Tempo: 133.355, Speechiness: 0.033, Loudness: -12.636',
 'Intruments: guitar saxophone, Tempo: 131.685, Speechiness: 0.0469, Loudness: -4.09',
 'Intruments: piano guitar, Tempo: 114.032, Speechiness: 0.0388, Loudness: -6.153',
 'Intruments: guitar drums, Tempo: 117.252, Speechiness: 0.07, Loudness: -7.16']

In [86]:

for match in response_matches:
    print(f"Letra: {match['metadata']['theme']}, \n Score: {match['score']}")

Letra: I’ve been hit by the bus of love That falling brick of you Baby, when there’s tears in your eyes You don’t have to lie, 
 Score: 0.200529367
Letra: Love and learn Take your turn Somewhere there's a feather Falling slowly from the sky, 
 Score: 0.192656934
Letra: The song "El Encuentro" by Biela explores themes of connection, longing, and intimacy. It captures the emotions surrounding a significant meeting between two individuals, emphasizing the beauty and complexity of human relationships. Through its lyrical imagery, it conveys a sense of urgency and the profound impact such encounters can have on one’s life. Overall, the song reflects on the joys and challenges of love and the deep bonds formed through shared experiences., 
 Score: 0.189094514
Letra: The theme of "Miles and Miles of Texas" revolves around the deep connection to the vast Texas landscape and the sense of freedom it brings to the narrator. It reflects a longing for home and the cherished memories tied to the pla

In [87]:
diff_chord_2= input_data['diff_chords_2_numeric'].iloc[0]
diff_chord_1= input_data['diff_chords_1_numeric'].iloc[0]

In [88]:
# Crear el embedding de la nueva query
chords_query = f"A song with a similar progession and distance (diff) in chords to {input_data['chords_1'][0]},y diff chords {diff_chord_2},{diff_chord_1}."
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=chords_query
).data[0].embedding

chords_query = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

chords_query_ = chords_query['matches']



In [89]:
for match in chords_query_:
    print(f"Chords: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}, \n Score: {match['score']}")

Chords: C G Am F, F Am C G, 
 Score: 0.473554373
Chords: C G Am F, C F G Am, 
 Score: 0.46341902
Chords: C G Am F, C G F F, 
 Score: 0.456121117
Chords: C G Am F, D A Bm G, 
 Score: 0.455776095
Chords: 2023 Remaster C G Am, F C G F, 
 Score: 0.449465752


In [90]:
chord_wheels=[]
for match in chords_query_:
    chord_wheels.append(f"Chord wheel: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}")

In [91]:
chord_wheels

['Chord wheel: C G Am F, F Am C G',
 'Chord wheel: C G Am F, C F G Am',
 'Chord wheel: C G Am F, C G F F',
 'Chord wheel: C G Am F, D A Bm G',
 'Chord wheel: 2023 Remaster C G Am, F C G F']

In [92]:
input_data['theme'][0]

'Suerte que en el sur hayas nacido Y que burlemos las distancias, Suerte que es haberte conocido Y por ti amar tierras extrañas Yo puedo escalar los andes solo Por ir a contar tus lunares,  Contigo celebro y sufro todo'

In [93]:
# Crear el embedding de la nueva query
lyrics_query = f"Give me a similar theme song to {input_data['theme'][0]}"
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=lyrics_query
).data[0].embedding

lyrics_result = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

lyrics_matches = lyrics_result['matches']



In [94]:
lyrics_list = []  # Crear una lista vacía para almacenar los textos

for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    lyrics_list.append(f"Lyrics: {match['metadata']['theme']}")



for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    print(f"Lyrics: {match['metadata']['theme']} {match['id']} Score: {match['score']}")

lyrics_list.append(input_data['theme'][0])

Lyrics: Suerte que en el sur hayas nacido Y que burlemos las distancias, Suerte que es haberte conocido Y por ti amar tierras extrañas Yo puedo escalar los andes solo Por ir a contar tus lunares,  Contigo celebro y sufro todo 3TY1PFZXmYeiLcXygB74My Score: 0.543976963
Lyrics: Dejare mi tierra por ti, Dejare mis campos y me iré Lejos de aquí, Cruzaré llorando el jardín y con tus recuerdos partiré, lejos de aquí  De día viviré pensando en tus sonrisas  5IU4JO2cpa0wPK8oQWFfCr Score: 0.486122221
Lyrics: The song "Cuando sale el sol" by Lágrimas De Sangre explores themes of resilience and hope, focusing on the rejuvenating power of new beginnings. It reflects on overcoming personal struggles and the importance of embracing life's challenges while maintaining optimism. The lyrics convey a sense of camaraderie and support, asserting that even in difficult times, the dawn brings new possibilities and a chance for renewal. Overall, it encapsulates the spirit of perseverance and the beauty of tra

In [95]:
lyrics_list

['Lyrics: Suerte que en el sur hayas nacido Y que burlemos las distancias, Suerte que es haberte conocido Y por ti amar tierras extrañas Yo puedo escalar los andes solo Por ir a contar tus lunares,  Contigo celebro y sufro todo',
 'Lyrics: Dejare mi tierra por ti, Dejare mis campos y me iré Lejos de aquí, Cruzaré llorando el jardín y con tus recuerdos partiré, lejos de aquí  De día viviré pensando en tus sonrisas ',
 'Lyrics: The song "Cuando sale el sol" by Lágrimas De Sangre explores themes of resilience and hope, focusing on the rejuvenating power of new beginnings. It reflects on overcoming personal struggles and the importance of embracing life\'s challenges while maintaining optimism. The lyrics convey a sense of camaraderie and support, asserting that even in difficult times, the dawn brings new possibilities and a chance for renewal. Overall, it encapsulates the spirit of perseverance and the beauty of transformation.',
 'Lyrics: Te conozco de otra vida, hoy saldrás por la vent

In [96]:
nueva_cancion = create_song(lyrics_list, chord_wheels, details)
print(nueva_cancion)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'top_p': decimal above maximum value. Expected a value <= 1, but got 1.1 instead.", 'type': 'invalid_request_error', 'param': 'top_p', 'code': 'decimal_above_max_value'}}

In [194]:
nueva_cancion

'# Mundo Repetido\n\n**Tempo:** 131 BPM  \n**Instrumentos principales:** Guitarra, Bajo\n\n### Letra y acordes:\n\n1. **Verso 1**  \n   [C] Salgo a la calle, [G] veo el mismo lugar,  \n   [Am] Rostros de plástico, [F] vuelven a chocar.  \n   [C] Cansado de cuentos, [G] que nadie escuchará,  \n   [Am] En casa me quedo, [F] nadie me encontrará.\n\n2. **Coro**  \n   [F] Dime por qué, [Am] este ciclo no termina,  \n   [C] Marchando al mismo [G] compás en la rutina.  \n   [F] Quiero volar, [Am] romper esta cadena,  \n   [C] La música que [G] suena, siempre tan ajena.\n\n3. **Verso 2**  \n   [C] Miradas que juzgan, [G] sin conocerme,  \n   [Am] Ideas que reinventan, [F] sin entenderme.  \n   [C] El mundo gira, [G] en esta monotonía,  \n   [Am] Busco escaparme, [F] sin tanta hipocresía.\n\n4. **Coro final**  \n   [F] Dime por qué, [Am] el reloj avanza lento,  \n   [C] Caminamos en [G] círculos como el viento.  \n   [F] Quiero volar, [Am] sentir la libertad,  \n   [C] La música se [G] apagará 